In [50]:
import sys
import os.path
import pikepdf
import urllib.request
import pandas as pd
from datetime import datetime
from pathlib import Path
Path("temp").mkdir(parents=True, exist_ok=True)

In [51]:
#data = pd.read_csv("temp/PDFURLs.csv", header=None)
#data

In [52]:
os.remove("temp/meta.csv")

In [53]:
#loop

int=1
with open("temp/PDFURLs.csv") as urls:
    for pdf_url in urls:

        unique = pdf_url.removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
        unique = unique.removesuffix("\n")
        pfad = "temp/" + unique + ".pdf"
        if (not os.path.exists(pfad)):
            print("downloading:", unique)
            pdf_filename, headers = urllib.request.urlretrieve(pdf_url, pfad)
            
        else:
            pdf_filename = pfad

        pdf = pikepdf.Pdf.open(pdf_filename)
        docinfo = pdf.docinfo
        metadict = {"Identifier":unique}



        try:
            docinfo["/Author"]
        except KeyError:
            metadict.update({"Author":""})
        else:
            metadict.update({"Author":str(docinfo["/Author"])})

        try:
            docinfo["/Publisher"]
        except KeyError:
            metadict.update({"Publisher":""})
        else:
            metadict.update({"Publisher":str(docinfo["/Publisher"])})

        try:
            docinfo["/CreationDate"]
        except KeyError:
            metadict.update({"Datum der Erstellung":""})
        else:
            creation = str(docinfo["/CreationDate"])
            date = datetime.strptime(creation.replace("'", ""), "D:%Y%m%d%H%M%S%z")
            metadict.update({"Datum der Erstellung":date.strftime("%x")}) #TODO: change to normal dates
    
        try:
            docinfo["/Title"]
        except KeyError:
            metadict.update({"Titel":""})
        else:
            metadict.update({"Titel":str(docinfo["/Title"])})
    
        metadict.update({"Seitenzahl":len(pdf.pages)})

        if (pdf.pages[0].mediabox[2] - pdf.pages[0].mediabox[0]) > (pdf.pages[0].mediabox[3] - pdf.pages[0].mediabox[1]):
            metadict.update({"Format":"Dokument"})
        else:
            metadict.update({"Format":"Präsentation"})
    
        size = os.stat(pdf_filename).st_size
        if (size > 1000000000):
            size = size / 1000000000
            sizestr =str(round(size, 1)) + " GB"
        elif (size > 1000000):                    
            size = size / 1000000
            sizestr =str(round(size, 1)) + " MB"
        elif (size > 1000):
            size = size / 1000
            sizestr =str(round(size, 1)) + " kB"   
        else:
            sizestr =str(round(size, 1)) + " B"
        metadict.update({"Dateigröße":str(sizestr)})

        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("temp/meta.csv")):   
            oldframe = pd.read_csv("temp/meta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("temp/meta.csv", index=False)
            
        else:
            frame.to_csv("temp/meta.csv", index=False)

        metadict.clear
        pdf.close()
        print("finshed:", int)
        int=int+1

finshed: 1
finshed: 2
finshed: 3
finshed: 4
finshed: 5
finshed: 6
finshed: 7
finshed: 8
finshed: 9
finshed: 10
finshed: 11
finshed: 12
finshed: 13
finshed: 14
finshed: 15
finshed: 16
finshed: 17
finshed: 18
finshed: 19
finshed: 20
finshed: 21
finshed: 22
finshed: 23
finshed: 24
finshed: 25
finshed: 26
finshed: 27
finshed: 28
finshed: 29
finshed: 30
finshed: 31
finshed: 32
finshed: 33
finshed: 34
finshed: 35
finshed: 36
finshed: 37
finshed: 38
finshed: 39
finshed: 40
finshed: 41
finshed: 42
finshed: 43
finshed: 44
finshed: 45
finshed: 46
finshed: 47
finshed: 48
